# Setup 

This notebook requires the following job(s) to be present in the database. 
```
615503
```

If you haven't done so yet, please **epmt submit** those directories to the database of your choice from the shell. See **README.md**:

```
$ epmt -v submit sample/615503.tgz
INFO:epmt_cmds:submit_to_db(sample/615503.tgz,*-papiex-[0-9]*-[0-9]*.csv,False)
INFO:epmt_cmds:./job_metadata is 6772 bytes in archive
INFO:epmt_cmds:1 files to submit
INFO:epmt_cmds:1 hosts found: ['pp053-collated-']
INFO:epmt_cmds:host pp053-collated-: 1 files to import
INFO:epmt_job:Binding to DB: {'filename': 'database.sqlite', 'create_db': True, 'provider': 'sqlite'}
INFO:epmt_job:Generating mapping from schema...
INFO:epmt_job:job_tags: {'atm_res': 'c96l49', 'exp_component': 'atmos', 'exp_name': 'ESM4_historical_D151', 'script_name': 'ESM4_historical_D151_atmos_18540101', 'ocn_res': '0.5l75', 'exp_time': '18540101'}
INFO:epmt_job:Processing job id 615503
INFO:epmt_job:Creating user Jeffrey.Durachta
INFO:epmt_job:Creating job 615503
INFO:epmt_job:Creating host pp053
INFO:epmt_job:Did 1000 (37734 in file)...236.49/sec
INFO:epmt_job:Did 2000 (37734 in file)...279.28/sec
INFO:epmt_job:Did 3000 (37734 in file)...293.66/sec
```

In [1]:
# Import the needed modules
import pandas as pd
import epmt_query as eq
import epmt_outliers as eod

{'host': 'localhost', 'password': 'example', 'user': 'postgres', 'dbname': 'EPMT', 'provider': 'postgres'}


In [2]:
jobs = eq.get_jobs(fmt='pandas')
print(jobs.columns.values)
display(jobs[['jobid', 'duration', 'cpu_time', 'num_procs']])

newdf = eod.detect_outlier_jobs(jobs)
print("Outlier jobs")
display(newdf)

[u'PERF_COUNT_SW_CPU_CLOCK' 'account' u'all_proc_tags'
 u'cancelled_write_bytes' 'cpu_time' u'delayacct_blkio_time' 'duration'
 'end' 'env_changes_dict' 'env_dict' 'exitcode' u'guest_time' u'inblock'
 'info_dict' u'invol_ctxsw' 'jobid' 'jobname' 'jobscriptname' u'majflt'
 u'minflt' u'num_procs' u'num_threads' u'outblock' 'ppr' u'processor'
 'queue' u'rchar' u'rdtsc_duration' u'read_bytes' u'rssmax' 'sessionid'
 'start' 'submit' u'syscr' u'syscw' u'systemtime' 'tags' u'time_oncpu'
 u'time_waiting' u'timeslices' 'updated_at' 'user' u'user+system'
 u'usertime' u'vol_ctxsw' u'wchar' u'write_bytes']


,jobid,duration,cpu_time,num_procs
0,615503,2.266020e+09,1.327237e+09,35483
1,kernel-build-20190610-081150,2.323364e+08,5.915147e+08,10601


Outlier jobs


,jobid,duration,cpu_time,num_procs
0,615503,False,False,False
1,kernel-build-20190610-081150,False,False,False


In [15]:
jobs = eq.get_jobs(fmt='terse')
print('jobs: ' + str(jobs))

tags = eq.get_unique_process_tags(exclude = ['op_instance', 'op_sequence', 'operation_count', 'instance'], fold=False)
print('tags:' + str(tags))
print

ops = eq.agg_metrics_by_tags(jobs=jobs,tags=tags,fmt='pandas')
print(ops.columns.values)

# Should be jobid
ops.rename(columns={'job':'jobid'},inplace=True)
                
display(ops[['jobid','tags', 'duration','exclusive_cpu_time','num_procs']])


newdfo = eod.detect_outlier_ops(ops)
print("Outlier operations")
display(newdfo)

jobs: [u'615503', u'kernel-build-20190610-081150']
tags:[{u'op': u'cp'}, {u'op': u'dmput'}, {u'op': u'fregrid'}, {u'op': u'hsmget'}, {u'op': u'mv'}, {u'op': u'ncatted'}, {u'op': u'ncks'}, {u'op': u'ncrcat'}, {u'op': u'plevel'}, {u'op': u'rm'}, {u'op': u'splitvars'}, {u'op': u'timavg'}, {u'op': u'untar'}, {u'operation': u'build'}, {u'operation': u'configure'}, {u'operation': u'download'}, {u'operation': u'extract'}]

[u'PERF_COUNT_SW_CPU_CLOCK' u'cancelled_write_bytes'
 u'delayacct_blkio_time' 'duration' 'exclusive_cpu_time' u'guest_time'
 u'inblock' u'invol_ctxsw' 'job' u'majflt' u'minflt' 'num_procs'
 'num_tids' u'outblock' u'processor' u'rchar' u'rdtsc_duration'
 u'read_bytes' u'rssmax' u'syscr' u'syscw' u'systemtime' 'tags'
 u'time_oncpu' u'time_waiting' u'timeslices' u'user+system' u'usertime'
 u'vol_ctxsw' u'wchar' u'write_bytes']


,jobid,tags,duration,exclusive_cpu_time,num_procs
0,615503,{u'op': u'cp'},1.224538e+08,40980648.0,3902
1,615503,{u'op': u'dmput'},2.286065e+09,12167869.0,272
2,615503,{u'op': u'fregrid'},3.997813e+08,396146734.0,38
3,615503,{u'op': u'hsmget'},2.863656e+09,270022419.0,15948
4,615503,{u'op': u'mv'},1.142269e+09,129375864.0,3393
5,615503,{u'op': u'ncatted'},3.300346e+07,33910501.0,3210
6,615503,{u'op': u'ncks'},5.172528e+07,45118505.0,720
7,615503,{u'op': u'ncrcat'},1.079935e+08,93014605.0,282
8,615503,{u'op': u'plevel'},2.084666e+08,105212285.0,2592
9,615503,{u'op': u'rm'},5.927420e+07,27965861.0,2772


Outlier operations


,jobid,duration,exclusive_cpu_time,num_procs
0,615503,False,False,False
1,615503,False,False,False
2,615503,False,False,False
3,615503,False,False,True
4,615503,False,False,False
5,615503,False,False,False
6,615503,False,False,False
7,615503,False,False,False
8,615503,False,False,False
9,615503,False,False,False


In [16]:
procs = eq.get_procs(jobs, fmt='pandas')
print(procs.columns.values)
print(procs['exename'].count())


olps = eod.detect_outlier_processes(procs)
display(procs[['id','exename','tags','duration','exclusive_cpu_time']].tail(20))
olps[['id','exename','tags','duration','exclusive_cpu_time']].tail(20)


[u'PERF_COUNT_SW_CPU_CLOCK' 'args' u'cancelled_write_bytes'
 u'delayacct_blkio_time' 'duration' 'end' 'exclusive_cpu_time' 'exename'
 'exitcode' 'gen' 'group' u'guest_time' 'host' 'id' u'inblock'
 'inclusive_cpu_time' u'invol_ctxsw' 'job' u'majflt' u'minflt' 'numtids'
 u'outblock' 'parent' 'path' 'pgid' 'pid' 'ppid' u'processor' u'rchar'
 u'rdtsc_duration' u'read_bytes' u'rssmax' 'sid' 'start' u'syscr' u'syscw'
 u'systemtime' 'tags' u'time_oncpu' u'time_waiting' u'timeslices'
 'updated_at' 'user' u'user+system' u'usertime' u'vol_ctxsw' u'wchar'
 u'write_bytes']
46084
('duration', array([   13,    71,    80, ..., 46081, 46082, 46083]))
('exclusive_cpu_time', array([   19,    71,    80, ..., 46074, 46081, 46082]))


,id,exename,tags,duration,exclusive_cpu_time
46064,35503,cat,"{u'instance': u'1', u'operation': u'configure'...",203.0,9616.0
46065,35502,bash,"{u'instance': u'1', u'operation': u'configure'...",926370.0,55178.0
46066,35501,make,"{u'instance': u'1', u'operation': u'configure'...",3926996.0,11593.0
46067,35500,make,"{u'instance': u'1', u'operation': u'configure'...",5645335.0,21705.0
46068,35499,dash,"{u'instance': u'1', u'operation': u'configure'...",11514012.0,11231.0
46069,35498,make,"{u'instance': u'1', u'operation': u'configure'...",11527394.0,10558.0
46070,35497,make,"{u'instance': u'1', u'operation': u'configure'...",11540720.0,12886.0
46071,35496,make,"{u'instance': u'1', u'operation': u'configure'...",13668007.0,28996.0
46072,35495,bash,"{u'instance': u'1', u'operation': u'build', u'...",134.0,3711.0
46073,35494,bash,"{u'instance': u'1', u'operation': u'build', u'...",12355.0,4073.0


,id,exename,tags,duration,exclusive_cpu_time
46064,35503,cat,"{u'instance': u'1', u'operation': u'configure'...",False,False
46065,35502,bash,"{u'instance': u'1', u'operation': u'configure'...",True,False
46066,35501,make,"{u'instance': u'1', u'operation': u'configure'...",True,False
46067,35500,make,"{u'instance': u'1', u'operation': u'configure'...",True,False
46068,35499,dash,"{u'instance': u'1', u'operation': u'configure'...",True,False
46069,35498,make,"{u'instance': u'1', u'operation': u'configure'...",True,False
46070,35497,make,"{u'instance': u'1', u'operation': u'configure'...",True,False
46071,35496,make,"{u'instance': u'1', u'operation': u'configure'...",True,False
46072,35495,bash,"{u'instance': u'1', u'operation': u'build', u'...",False,False
46073,35494,bash,"{u'instance': u'1', u'operation': u'build', u'...",False,False
